<a href="https://colab.research.google.com/github/Xelszy/stablebreaktest/blob/main/stable_diffusion_breaktherules_nomodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output, display, HTML
import os
import time
import subprocess
from datetime import timedelta
from google.colab import drive
from IPython.utils import capture
from subprocess import getoutput
from urllib.parse import unquote
from google.colab.output import eval_js
%cd /content
%env TF_CPP_MIN_LOG_LEVEL=1

try:
    start_colab
except:
    start_colab = int(time.time()) - 5
clear_output()
print("\033[92m") 
verbose_download = True #@param {type:"boolean"} 
try:
  output 
except:
  print('‚åö Checking GPU...', end='')
  output = getoutput('nvidia-smi --query-gpu=gpu_name --format=csv')
  if "name" in output:
    gpu_name = output[5:]
    print('\r‚úÖ Current GPU:', gpu_name, flush=True)
  else:
    print('\r\033[91m‚ùé ERROR: No GPU detected. Please do step below to enable.\n', flush=True)
    display(HTML("<img src='https://i.ibb.co/HC9KH17/NVIDIA-Share-23-01-02-173037.png' width='800px'/>"))
    print('\033[91m\nIf it says "Cannot connect to GPU backend", meaning you\'ve either reached free usage limit. OR there\'s no gpu available.\n\nDon\'t mind me... I\'m destroying your current session for your own good...')
    display(HTML("<img src='https://media.tenor.com/E9omRGF7x0AAAAAC/hitori-gotou-bocchi-rock.gif' width='500px'/>"))
    time.sleep(5)
    from google.colab import runtime
    runtime.unassign()

#Configuration
if verbose_download:
  if not os.path.exists('/content/breaktherules'):
    print("üöÄ Downloading...")
    !apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev > /dev/null 2>&1
    !pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U > /dev/null 2>&1
    !pip install xformers==0.0.19 triton==2.0.0 -U > /dev/null 2>&1
    !git clone --quiet https://github.com/Xelszy/stablebreaktest /content/breaktherules
    !apt-get install -y aria2 > /dev/null
    !apt-get install -y ripgrep > /dev/null
else:
    print("üìÇ Folder '/content/breaktherules' already exists. Skipping download.")

print("üöÄ Unpacking... Please do not stop this process at all cost...")
 
process = subprocess.Popen(["python", "/content/breaktherules/stablebreaktest.py"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, error = process.communicate()
print("All done!")

# Path Set
destination_dir = "/content/.downloaded/"
config_dir="/content/stablebreaktest/config.json"
models_dir = "/content/stablebreaktest/models/Stable-diffusion/"
vaes_dir = "/content/stablebreaktest/models/VAE/"
hypernetworks_dir = "/content/stablebreaktest/models/hypernetworks/"
embeddings_dir = "/content/stablebreaktest/embeddings/"
loras_dir = "/content/stablebreaktest/models/Lora"
lycoris_dir = "/content/stablebreaktest/models/LyCORIS"
patches_dir = "/content/patches"
extensions_dir = "/content/stablebreaktest/extensions/"
control_dir = "/content/stablebreaktest/extensions/sd-webui-controlnet/models"
commandline_args = "--listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple --no-hashing --no-half-vae" #@param {type:"string"}

# Unpack Repo, Dependencies, Caches
if not os.path.exists("/content/stablebreaktest"):
  start_install = int(time.time())
  print("üöÄ Unpacking... Please do not stop this process at all cost...", end='')
  with capture.capture_output() as cap:
    !wget https://huggingface.co/NoCrypt/fast-repo/resolve/main/ubuntu_deps.zip && unzip -o ubuntu_deps.zip -d ./deps && dpkg -i ./deps/* && rm -rf ubuntu_deps.zip /content/stablebreaktest/
    !echo -e "https://huggingface.co/NoCrypt/fast-repo/resolve/main/dep.tar.lz4\n\tout=dep.tar.lz4\nhttps://huggingface.co/NoCrypt/fast-repo/resolve/main/repo.tar.lz4\n\tout=repo.tar.lz4\nhttps://huggingface.co/NoCrypt/fast-repo/resolve/main/cache.tar.lz4\n\tout=cache.tar.lz4\n" | \
    aria2c -i- -j5 -x16 -s16 -k1M -c 

    !tar -xI lz4 -f dep.tar.lz4 --overwrite-dir --directory=/usr/local/lib/python3.10/dist-packages/
    !tar -xI lz4 -f repo.tar.lz4 --directory=/
    !tar -xI lz4 -f cache.tar.lz4 --directory=/

    !rm -rf /content/dep.tar.lz4 /content/repo.tar.lz4 /content/cache.tar.lz4
    os.environ["SAFETENSORS_FAST_GPU"]='1'
    os.environ["colab_url"] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
    os.environ["PYTHONWARNINGS"] = "ignore"
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "garbage_collection_threshold:0.9,max_split_size_mb:512"

    %env LD_PRELOAD=libtcmalloc.so

    install_time = timedelta(seconds=time.time() - start_install)
    print("\rüöÄ Finished unpacking. Took","%02d:%02d:%02d ‚ö°\n" % (install_time.seconds / 3600, (install_time.seconds / 60) % 60, install_time.seconds % 60), end='', flush=True)
     
# NoCrypt Colab Timer
    print("ü§ù Colab timer integration complete! You can see your colab time inside webui.")

print('\rüèÅ Download finished.', flush=True)

# Update 
print('‚åö Updating extensions (might take a while)...', end="")
with capture.capture_output() as cap:
    !for dir in /content/stablebreaktest/extensions/*/; do cd "$dir" && git fetch origin && git pull; done
    del cap
print('\rü™Ñ \033[92mInbuilt extensions are updated to their latest versions', flush=True)
time_since_start = timedelta(seconds=time.time() - start_colab)
print("‚åö You've been running this Colab for","%02d:%02d:%02d" % (time_since_start.seconds / 3600, (time_since_start.seconds / 60) % 60, time_since_start.seconds % 60))

#Just Moving
!mkdir -p /content/stablebreaktest/static
!touch /content/stablebreaktest/static/colabTimer.txt
!mv /content/breaktherules/setuptimer.py /content/stablebreaktest
!mv /content/breaktherules/colab-timer.js /content/stablebreaktest

# Colab ü§ù Gradio (Colab timer integration for gradio)
!echo -n {start_colab} > /content/stablebreaktest/static/colabTimer.txt
print("ü§ù Colab timer integration complete! You can see your colab time inside webui.")

# Print all files in every important directory
!echo -e "\033[92mCan't see your files in here? Activate verbose_download to debug!\n"
print("\033[92m\033[1m‚ï≠-üì¶ Models + Configs\033[96m")
!find {models_dir}/ -mindepth 1 ! -name '*.txt' -printf '%f\n' 
print("\n\033[92m\033[1m‚ï≠-üì¶ VAEs\033[96m")
!find {vaes_dir}/ -mindepth 1 ! -name '*.txt' -printf '%f\n'
print("\n\033[92m\033[1m‚ï≠-üì¶ Custom Embeddings (inbuilt is hidden)\033[96m")
!find {embeddings_dir}/ -mindepth 1 -maxdepth 1 -name '*.pt' -or -name '*.safetensors' -printf '%f\n'
print("\n\033[92m\033[1m‚ï≠-üì¶ LoRAs\033[96m")
!find {loras_dir}/ -mindepth 1 ! -name '*.keep' -printf '%f\n'
print("\n\033[92m\033[1m‚ï≠-üì¶ LyCORIS\033[96m")
!find {lycoris_dir}/ -mindepth 1 ! -name '*.keep' -printf '%f\n'
print("\n\033[92m\033[1m‚ï≠-üì¶ Hypernetworks\033[96m")
!find {hypernetworks_dir}/ -mindepth 1 ! -name '*.txt' -printf '%f\n'
print("\n\033[92m\033[1m‚ï≠-üì¶ Extensions\033[96m")
!find {extensions_dir}/ -mindepth 1 -maxdepth 1 ! -name '*.txt' -printf '%f\n'
print("\n\n\033[0m")

# Start the webui
%cd /content/stablebreaktest
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/stablebreaktest/models/Stable-diffusion -o vae-ft.vae.pt

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev > /dev/null 2>&1
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U > /dev/null 2>&1
!pip install -q xformers==0.0.16 triton==2.0.0 -U > /dev/null 2>&1

!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stablebreaktest/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stablebreaktest/launch.py
!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' /content/stablebreaktest/modules/shared.py

!python /content/stablebreaktest/setuptimer.py
!wget -q https://raw.githubusercontent.com/etherealxx/batchlinks-webui/main/notification.mp3 -P /content/stablebreaktest
!echo -n {start_colab} > /content/stablebreaktest/static/colabTimer.txt

!COMMANDLINE_ARGS="{commandline_args}" REQS_FILE="requirements_versions.txt" python launch.py 
time_since_start = timedelta(seconds=time.time()-start_colab)
print("\n\n\033[96m‚åö You've been running this colab for","%02d:%02d:%02d" % (time_since_start.seconds / 3600, (time_since_start.seconds / 60) % 60, time_since_start.seconds % 60))
print("\n\n")